In [ ]:
!pip install --upgrade nibabel

Get data


In [ ]:
!wget "https://putpoznanpl-my.sharepoint.com/:u:/g/personal/dominik_pieczynski_put_poznan_pl/EWIZ_xm8wXpMjQDgF2VQ1csB4QuHPKoj5vDpj6CQi9p-AA?e=yQr6fn&download=1" -O public.zip
!unzip -q public.zip
!rm public.zip

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


Create valid dirs

In [ ]:
!mkdir /content/FirstDataset/valid
!mkdir /content/SecondDataset/valid

In [ ]:
!mkdir /content/FirstDataset/train2

Checking the number of scans in dirs 

In [ ]:
import os
import shutil

print("FirstDataset")
filepath1 = '/content/FirstDataset/train'
filenames1 = os.listdir(filepath1)
filenames1 = sorted(filenames1)
print('train', len(filenames1))
print('test', len(os.listdir('/content/FirstDataset/test')))
print(int(len(filenames1)/5)) #114

print("SecondDataset")
filepath2 = '/content/SecondDataset/train'
filenames2 = os.listdir(filepath2)
filenames2 = sorted(filenames2)
print('train', len(filenames2))
print('test', len(os.listdir('/content/SecondDataset/test')))
print(int(len(filenames2)/5)) # 20

FirstDataset
train 574
test 72
114
SecondDataset
train 100
test 25
20


In [ ]:
import os
import shutil
from pathlib import Path

# FirstDataset
filepath1 = Path('/content/FirstDataset/train')

train_X_1 = []
train_y_1 = []

for scan_path in sorted(filepath1.iterdir()):
  if scan_path.name.endswith('mask.nii.gz'):
    train_y_1.append(scan_path)
  else:
    train_X_1.append(scan_path)

print(len(train_X_1), train_X_1)
print(len(train_y_1), train_y_1)

# SecondDataset
filepath2 = Path('/content/SecondDataset/train')

train_X_2 = []
train_y_2 = []

for scan_path in sorted(filepath2.iterdir()):
    train_y_2.append(scan_path/'mask.nii.gz')
    train_X_2.append(scan_path/'T1w.nii.gz')

print(len(train_X_2), train_X_2)
print(len(train_y_2), train_y_2)



287 [PosixPath('/content/FirstDataset/train/011e40c2b1d3c9bb53bd9e5a7efd04ba.nii.gz'), PosixPath('/content/FirstDataset/train/0187e4cb2fb4971848c8981e9ece0849.nii.gz'), PosixPath('/content/FirstDataset/train/022161680299e1f3bea7169947dc45b6.nii.gz'), PosixPath('/content/FirstDataset/train/02862555e24fd75a84a3259e4210e73b.nii.gz'), PosixPath('/content/FirstDataset/train/04e0e6ed0432fd737fd3f29fe551b77c.nii.gz'), PosixPath('/content/FirstDataset/train/05b438c4933b77168c529ea19486a1c9.nii.gz'), PosixPath('/content/FirstDataset/train/06503d8025b2830c50599cfcc0febd39.nii.gz'), PosixPath('/content/FirstDataset/train/0a2de3cae8ca66ea62b7e2dae3bdffca.nii.gz'), PosixPath('/content/FirstDataset/train/0b3ea09bb26b17af2b596e96875d2ca2.nii.gz'), PosixPath('/content/FirstDataset/train/0c7dc995949da4278941d36c048ff2c9.nii.gz'), PosixPath('/content/FirstDataset/train/0efee62646deb97c8c7cfa0f457fa4dc.nii.gz'), PosixPath('/content/FirstDataset/train/105625ce5c645192e3c3ca78b7b32edb.nii.gz'), PosixPath('

In [ ]:
from sklearn.model_selection import train_test_split

X_train_1, X_valid_1, y_train_1, y_valid_1 = train_test_split(train_X_1, train_y_1, train_size=0.9, random_state=42)
X_train_2, X_valid_2, y_train_2, y_valid_2 = train_test_split(train_X_2, train_y_2, train_size=0.9, random_state=42)

X_train = X_train_1 + X_train_2
y_train = y_train_1 + y_train_2
X_valid = X_valid_1 + X_valid_2
y_valid = y_valid_1 + y_valid_2

print('X_train len:', len(X_train))
print('y_train len:', len(y_train))
print('X_valid len:', len(X_valid))
print('y_valid len:', len(y_valid))


X_train len: 348
y_train len: 348
X_valid len: 39
y_valid len: 39


###Eksport to png

Template methods

In [17]:
import numpy as np
import nibabel as nib

from typing import Tuple, List
from pathlib import Path


def load_raw_volume(path: Path) -> Tuple[np.ndarray, np.ndarray]:
  data: nib.Nifti1Image = nib.load(str(path))
  data = nib.as_closest_canonical(data)
  raw_data = data.get_fdata(caching='unchanged', dtype=np.float32)
  return raw_data, data.affine


def load_labels_volume(path: Path) -> np.ndarray:
  return load_raw_volume(path)[0].astype(np.uint8)


def save_labels(data: np.ndarray, affine: np.ndarray, path: Path):
  nib.save(nib.Nifti1Image(data, affine), str(path))


def show_slices(slices: List[np.ndarray]):
   fig, axes = plt.subplots(1, len(slices))
   for i, data_slice in enumerate(slices):
       axes[i].imshow(data_slice.T, cmap="gray", origin="lower")

Create dirs for images

In [ ]:
!rm -rf /content/Images

!mkdir /content/Images

!mkdir /content/Images/valid
!mkdir /content/Images/valid/images
!mkdir /content/Images/valid/images/images
!mkdir /content/Images/valid/masks
!mkdir /content/Images/valid/masks/masks

!mkdir /content/Images/train
!mkdir /content/Images/train/images
!mkdir /content/Images/train/images/images
!mkdir /content/Images/train/masks
!mkdir /content/Images/train/masks/masks

print("done")

done


In [ ]:
!ls /content/Images/valid/images/images

Export to png

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import cv2
import numpy as np


image_train_path = '/content/Images/train/images/images/'
mask_train_path = '/content/Images/train/masks/masks/'

image_valid_path = '/content/Images/valid/images/images/'
mask_valid_path = '/content/Images/valid/masks/masks/'


print('X_train start')
for scan_idx, scan_path in enumerate(X_train):
  print(scan_path)
  if scan_path.name.endswith('mask.nii.gz'):
    pass
  else:
    raw_volume, affine = load_raw_volume(scan_path)
    volume_max = np.amax(raw_volume)
    volume_min = np.amin(raw_volume)

    for slice_idx in range(raw_volume.shape[0]):
      path = f'{image_train_path}train_image_{scan_idx}_slice_{slice_idx}_.png'
      # print(path)

      volume = (raw_volume[slice_idx]-volume_min)/(volume_max-volume_min)
      volume = volume*255
      volume = volume.astype('int32')
      matplotlib.image.imsave(path, volume, vmin=0,vmax=255)
print("X_train finish")

print('y_train start')
for scan_idx, scan_path in enumerate(y_train):
  print(scan_path)

  if scan_path.name.endswith('mask.nii.gz'):
    mask_volume = load_labels_volume(scan_path)
    for slice_idx in range(mask_volume.shape[0]):

      path = f'{mask_train_path}train_mask_{scan_idx}_slice_{slice_idx}_.png'
      matplotlib.image.imsave(path, mask_volume[slice_idx],cmap="gray")

  else:
    pass
print("y_train finish")

print('X_valid start')
for scan_idx, scan_path in enumerate(X_valid):
  print(scan_path)
  if scan_path.name.endswith('mask.nii.gz'):
    pass
  else:
    raw_volume, affine = load_raw_volume(scan_path)
    volume_max = np.amax(raw_volume)
    volume_min = np.amin(raw_volume)

    for slice_idx in range(raw_volume.shape[0]):
      path = f'{image_valid_path}valid_image_{scan_idx}_slice_{slice_idx}_.png'
      # print(path)

      volume = (raw_volume[slice_idx]-volume_min)/(volume_max-volume_min)
      volume = volume*255
      volume = volume.astype('int32')
      matplotlib.image.imsave(path, volume, vmin=0,vmax=255)
print("X_valid finish")

print('y_valid start')
for scan_idx, scan_path in enumerate(y_valid):
  print(scan_path)

  if scan_path.name.endswith('mask.nii.gz'):
    mask_volume = load_labels_volume(scan_path)
    for slice_idx in range(mask_volume.shape[0]):

      path = f'{mask_valid_path}train_mask_{scan_idx}_slice_{slice_idx}_.png'
      matplotlib.image.imsave(path, mask_volume[slice_idx],cmap="gray")

  else:
    pass
print("y_valid finish")



In [ ]:
!ls /content/Images/train/images/images/


In [ ]:
import os
import shutil
from pathlib import Path

!zip -r data_images2.zip /content/Images
src  = '/content/data_images2.zip'
dst = '/content/drive/MyDrive/ZPO_projekt'

shutil.move(src,dst)

In [ ]:
!unzip '/content/drive/MyDrive/ZPO_projekt/data_images2.zip'

In [ ]:
!ls /content/content/Images/train/images/images

In [5]:
import tensorflow as tf
# from segmentation_models import get_preprocessing

# preprocesing_function = get_preprocessing('efficientnetb1')

augmentations=dict(
        # horizontal_flip=True,
        width_shift_range=0.1,
        height_shift_range=0.1,
        rotation_range=5
)

train_images_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1/255,
    # **augmentations
    # preprocessing_function=preprocesing_function,
    )

train_mask_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1/255,
    # **augmentations
    # preprocessing_function=preprocesing_function,
 )

In [6]:
import tensorflow as tf
# from segmentation_models import get_preprocessing


batch_size = 128
# preprocesing_function = get_preprocessing('efficientnetb0')


train_images_generator = train_images_datagen.flow_from_directory(
    '/content/content/Images/train/images',
    target_size=(128, 128),
    batch_size=batch_size,
    seed=42,
    interpolation='bilinear',
    color_mode='grayscale',
    class_mode=None,
    # subset = 'training'
    )



train_mask_generator = train_mask_datagen.flow_from_directory(
    '/content/content/Images/train/masks',
     target_size=(128, 128),
     batch_size=batch_size,
     seed=42,
     class_mode=None,
     color_mode='grayscale',
     interpolation='nearest',
    #  subset = 'training'
    )


train_combined_generator = zip(train_images_generator, train_mask_generator)





Found 69503 images belonging to 1 classes.
Found 69503 images belonging to 1 classes.


In [7]:

valid_images_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    # preprocessing_function=preprocesing_function,
    rescale = 1/255,
    )

valid_mask_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    # preprocessing_function=preprocesing_function,
    rescale = 1/255,
 )

In [8]:

val_images_generator = valid_images_datagen.flow_from_directory(
    '/content/content/Images/valid/images',
    target_size=(64, 64),
    batch_size=batch_size,
    seed=42,
    interpolation='bilinear',
    color_mode='grayscale',
    class_mode=None
    # subset = 'validation'
    )



val_mask_generator = valid_mask_datagen.flow_from_directory(
    '/content/content/Images/valid/masks',
     target_size=(64, 64),
     batch_size=batch_size,
     interpolation ='nearest',
     color_mode='grayscale',
     seed=42,
     class_mode=None
    #  subset = 'validation'
    )
val_combined_generator = zip(val_images_generator, val_mask_generator)

Found 7712 images belonging to 1 classes.
Found 7712 images belonging to 1 classes.


In [ ]:
joined_generator = zip(train_images_generator, train_mask_generator)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

images,masks = next(joined_generator)

fig = plt.figure()
fig.add_subplot(1, 2, 1)
plt.imshow(images[0].astype(int))
fig.add_subplot(1, 2, 2)
plt.imshow(masks[0].astype(int))

In [ ]:
# segmentation_models nie jest domyślnie zainstalowane w Colabie
!pip install segmentation_models

In [ ]:
%env SM_FRAMEWORK=tf.keras

In [11]:
def dice_coef(y_true, y_pred, smooth=0.001):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """


    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    # print((2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth))
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)

 
def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

In [12]:
import segmentation_models
import keras
from segmentation_models import Unet
from segmentation_models.losses import DiceLoss

from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
# import segmentation_models. as smp

model_base = Unet('efficientnetb1', classes=1, activation='sigmoid', encoder_weights='imagenet', encoder_freeze=False)


inp = Input(shape=(None, None, 1))


l1 = tf.keras.layers.Conv2D(3, (1, 1))(inp) # map N channels data to 3 channels
out = model_base(l1)
model = Model(inp, out, name="model")

optimizer = tf.keras.optimizers.Adam(lr=2e-4)
loss =  DiceLoss()
model.compile(optimizer=optimizer,
                   loss=loss,
                   metrics=[dice_coef,keras.metrics.BinaryAccuracy(),
                            segmentation_models.metrics.IOUScore(threshold=0.5),
                            segmentation_models.metrics.FScore(threshold=0.5)])


Segmentation Models: using `tf.keras` framework.
27164672/27164032 [==============================] - 2s 0us/step


In [ ]:
model.summary()

In [13]:
from keras.callbacks import ModelCheckpoint


training_samples = train_images_generator.n
validation_samples = val_images_generator.n


batch_size = 128


model.fit(
    train_combined_generator, 
    steps_per_epoch=training_samples // batch_size,
    validation_data = val_combined_generator,
    validation_steps = validation_samples // batch_size,
    epochs=2,
    workers=2,
    use_multiprocessing=True,
    verbose=1,
)
model.save('/content/drive/MyDrive/ZPO_projekt/model5.hdf5')

Epoch 1/2
542/542 [==============================] - 369s 642ms/step - loss: 0.3963 - dice_coef: 0.7855 - binary_accuracy: 0.8963 - iou_score: 0.7341 - f1-score: 0.8043 - val_loss: 0.3546 - val_dice_coef: 0.6951 - val_binary_accuracy: 0.9484 - val_iou_score: 0.5057 - val_f1-score: 0.6713
Epoch 2/2
542/542 [==============================] - 326s 602ms/step - loss: 0.0333 - dice_coef: 0.9853 - binary_accuracy: 0.9962 - iou_score: 0.9639 - f1-score: 0.9816 - val_loss: 0.1972 - val_dice_coef: 0.8209 - val_binary_accuracy: 0.9665 - val_iou_score: 0.6773 - val_f1-score: 0.8073


In [ ]:
# model.save('/content/drive/MyDrive/ZPO_projekt/model2.hdf5')

In [24]:
!rm -r /content/Predictions/

In [25]:
from pathlib import Path

predictions_base_path = Path('/content/Predictions')
first_dataset_predictions_path = predictions_base_path / 'first'
second_dataset_predictions_path = predictions_base_path / 'second'

first_dataset_predictions_path.mkdir(exist_ok=True, parents=True)
second_dataset_predictions_path.mkdir(exist_ok=True, parents=True)

In [ ]:
model.load_weights(f"/content/drive/MyDrive/ZPO_projekt/model3.hdf5")

In [ ]:
import cv2
import matplotlib.pyplot as plt


first_dataset_test_path = Path('/content/FirstDataset/test')
second_dataset_test_path = Path('/content/SecondDataset/test')

licznik = 1
for scan_path in first_dataset_test_path.iterdir():
  print(licznik)
  licznik += 1

  data, affine = load_raw_volume(scan_path)
  labels = np.zeros(data.shape, dtype=np.uint8)

  data_max=np.amax(data)
  data_min=np.amin(data)

  x_size, y_size, z_size = data.shape
  for x_index in range(x_size):
    data_slice = data[x_index]
    # normalizacja
    data_slice = (data_slice - data_min)/(data_max - data_min)
    # zmiana wymiaru do wymiaru na którym była uczona sieć
    data_slice = cv2.resize(data_slice, (128, 128))
    data_slice = data_slice[...,np.newaxis]
    data_slice = data_slice[np.newaxis,...]

    prediction = model.predict(data_slice)
    prediction = prediction.squeeze()
    # plt.imshow(prediction)
    # plt.show()
    # binaryzacja ( zostają wartości 0 oraz 1)
    prediction[prediction<0.05]=0
    prediction[prediction>=0.05]=1
    # plt.imshow(prediction)
    # plt.show()
    prediction = cv2.resize(prediction,(z_size,y_size))

    labels[x_index] = prediction

  save_labels(labels, affine, first_dataset_predictions_path / scan_path.name)

#----------------------------------------------
licznik = 1 
for scan_path in second_dataset_test_path.iterdir():
  print(licznik)
  licznik += 1

  data, affine = load_raw_volume(scan_path / 'T1w.nii.gz')
  labels = np.zeros(data.shape, dtype=np.uint8)
  
  data_max=np.amax(data)
  data_min=np.amin(data)

  x_size, y_size, z_size = data.shape
  for x_index in range(x_size):
    data_slice = data[x_index]
    # normalizacja
    data_slice = (data_slice - data_min)/(data_max - data_min)
    # zmiana wymiaru do wymiaru na którym była uczona sieć
    data_slice = cv2.resize(data_slice, (128, 128))
    data_slice = data_slice[...,np.newaxis]
    data_slice = data_slice[np.newaxis,...]
    prediction = model.predict(data_slice)
    prediction = prediction.squeeze()
    # plt.imshow(prediction)
    # plt.show()
    # binaryzacja ( zostają wartości 0 oraz 1)
    prediction[prediction<0.05]=0
    prediction[prediction>=0.05]=1
    # plt.imshow(prediction)
    # plt.show()
    prediction = cv2.resize(prediction,(z_size,y_size))

    labels[x_index] = prediction
  
  save_labels(labels, affine, second_dataset_predictions_path / f'{scan_path.name}.nii.gz')

In [ ]:
import os
import shutil
from pathlib import Path

!zip -r predictions_3.zip /content/Predictions/
src  = '/content/predictions_3.zip'
dst = '/content/drive/MyDrive/ZPO_projekt'

shutil.move(src,dst)

In [ ]:
import requests
import zlib

score = []
for dataset_predictions_path in (first_dataset_predictions_path, second_dataset_predictions_path):
  for prediction_path in dataset_predictions_path.iterdir():
    prediction_name = prediction_path.name[:-7]  # Usuwanie '.nii.gz' z nazwy pliku
    prediction = nib.load(str(prediction_path))

    response = requests.post(f'https://zpo.dpieczynski.pl/{prediction_name}', data=zlib.compress(prediction.to_bytes()))
    if response.status_code == 200:
        print(dataset_predictions_path.name, prediction_path.name, response.json())
        score.append(response.json()['dice'])
    else:
        print(f'Error processing prediction {dataset_predictions_path.name}/{prediction_name}: {response.text}')
print("mean score:", np.mean(score))